In [9]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [10]:
import re
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters

# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import random_search_forecaster
from skforecast.model_selection import bayesian_search_forecaster

from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler

In [11]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()

# Train-val-test dates
# ==============================================================================
end_train = '2001-01-01 23:59:00'
end_val = '2006-01-01 23:59:00'

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


In [12]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 10 # Placeholder, the value will be overwritten
             )

# Lags used as predictors
lags_grid ={
    'lags_1': 3,
    'lags_2': 10,
    'lags_3': [1, 2, 3, 20]
}

# Regressor hyperparameters
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [2, 3, 5]
}


results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = data.loc[:end_val, 'y'],
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = ['mean_squared_error', 'mean_absolute_error'],
              initial_train_size = len(data.loc[:end_train]),
              method             = 'backtesting',
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = False,
              show_progress      = True
          )

results

Number of models compared: 18.


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/6 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3 20] 
  Parameters: {'max_depth': 2, 'n_estimators': 100}
  Backtesting metric: 0.043298239712353144



,lags,lags_label,params,mean_squared_error,mean_absolute_error,max_depth,n_estimators
0,"[1, 2, 3, 20]",lags_3,"{'max_depth': 2, 'n_estimators': 100}",0.043298,0.182403,2,100
1,"[1, 2, 3]",lags_1,"{'max_depth': 3, 'n_estimators': 100}",0.043843,0.183544,3,100
2,"[1, 2, 3]",lags_1,"{'max_depth': 5, 'n_estimators': 100}",0.043875,0.183594,5,100
3,"[1, 2, 3, 20]",lags_3,"{'max_depth': 3, 'n_estimators': 100}",0.044074,0.184901,3,100
4,"[1, 2, 3, 20]",lags_3,"{'max_depth': 5, 'n_estimators': 100}",0.044074,0.184901,5,100
5,"[1, 2, 3]",lags_1,"{'max_depth': 2, 'n_estimators': 100}",0.044107,0.183931,2,100
6,"[1, 2, 3]",lags_1,"{'max_depth': 3, 'n_estimators': 50}",0.045422,0.186143,3,50
7,"[1, 2, 3]",lags_1,"{'max_depth': 5, 'n_estimators': 50}",0.045423,0.186145,5,50
8,"[1, 2, 3]",lags_1,"{'max_depth': 2, 'n_estimators': 50}",0.045827,0.186807,2,50
9,"[1, 2, 3, 20]",lags_3,"{'max_depth': 3, 'n_estimators': 50}",0.046221,0.187184,3,50


In [13]:
results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = data.loc[:end_val, 'y'],
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = ['mean_squared_error', 'mean_absolute_error'],
              initial_train_size = len(data.loc[:end_train]),
              method             = 'one_step_ahead',
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = True,
              show_progress      = True
          )

results

Number of models compared: 18.
Models are evaluated using one-step-ahead predictions. This is faster than backtesting the model to predict multiple steps ahead and is useful for quickly comparing different models and hyperparameters, but the results may not reflect the actual performance of the model when predicting multiple steps ahead.


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/6 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'max_depth': 5, 'n_estimators': 100}
  Backtesting metric: 0.04087015439759103



,lags,lags_label,params,mean_squared_error,mean_absolute_error,max_depth,n_estimators
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 5, 'n_estimators': 100}",0.040870,0.159246,5,100
1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 2, 'n_estimators': 100}",0.041841,0.165568,2,100
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 5, 'n_estimators': 50}",0.042680,0.166428,5,50
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 3, 'n_estimators': 100}",0.043492,0.166654,3,100
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 2, 'n_estimators': 50}",0.043770,0.170828,2,50
5,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",lags_2,"{'max_depth': 3, 'n_estimators': 50}",0.044719,0.171036,3,50
6,"[1, 2, 3, 20]",lags_3,"{'max_depth': 5, 'n_estimators': 50}",0.045570,0.174160,5,50
7,"[1, 2, 3, 20]",lags_3,"{'max_depth': 3, 'n_estimators': 50}",0.046647,0.176595,3,50
8,"[1, 2, 3]",lags_1,"{'max_depth': 5, 'n_estimators': 100}",0.047074,0.178880,5,100
9,"[1, 2, 3, 20]",lags_3,"{'max_depth': 3, 'n_estimators': 100}",0.047183,0.172072,3,100


## Benchmarking

In [14]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from skforecast.datasets import fetch_dataset

# Modelado y Forecasting
# ==============================================================================
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
import shap

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

In [15]:
data = fetch_dataset('bike_sharing_extended_features')

# Split train-validation-test
# ==============================================================================
end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]

print(f"Dates train      : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Dates validacion : {data_val.index.min()} --- {data_val.index.max()}  (n={len(data_val)})")
print(f"Dates test       : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

exog_features = [
    'month_sin', 
    'month_cos',
    'week_of_year_sin',
    'week_of_year_cos',
    'week_day_sin',
    'week_day_cos',
    'hour_day_sin',
    'hour_day_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day',
    'temp',
    'holiday'
]

bike_sharing_extended_features
------------------------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, the dataset
was enriched by introducing supplementary features. Addition includes calendar-
based variables (day of the week, hour of the day, month, etc.), indicators for
sunlight, incorporation of rolling temperature averages, and the creation of
polynomial features generated from variable pairs. All cyclic variables are
encoded using sine and cosine functions to ensure accurate representation.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17352, 90)
Dates train      : 2011-01-08 00:00:00 --- 2012-03-31 23:00:00  (n=10776)
Dates validacion : 2012-04-01 00:00:00 --- 2012-08-31 23:00:00  (n=3672)
Dates test       : 2012-09-01 00:00:00 --- 2012-12-30 23:00:00  (n=2904)


In [16]:
# Create forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 24
             )

In [19]:
# Búsqueda de hiperparámetros
# ==============================================================================
# Lags candidatos
lags_grid = [48, 72, (1, 2, 3, 23, 24, 25, 167, 168, 169)]

# grid de búsqueda de hiperparámetros

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 5],
    'learning_rate': [0.01, 0.1, 0.5]
}

results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = data.loc[:end_validation, 'users'],
              exog               = data.loc[:end_validation, exog_features],
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = ['mean_squared_error', 'mean_absolute_error'],
              method             = 'backtesting',
              initial_train_size = len(data.loc[:end_train]),
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = False,
              show_progress      = True
          )

display(results.head(5))

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = data['users'],
    exog               = data[exog_features],
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data[:end_validation]),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
metric

Number of models compared: 81.


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/27 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
  Backtesting metric: 8229.839944707848



,lags,lags_label,params,mean_squared_error,mean_absolute_error,learning_rate,max_depth,n_estimators
0,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",8229.839945,58.574818,0.1,3.0,200.0
1,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",8364.261661,59.013335,0.1,3.0,100.0
2,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'learning_rate': 0.1, 'max_depth': 2, 'n_esti...",8476.699921,59.764132,0.1,2.0,200.0
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",8770.705516,61.775697,0.1,5.0,100.0
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",8962.650047,62.304379,0.1,5.0,200.0


  0%|          | 0/81 [00:00<?, ?it/s]

,mean_absolute_error
0,57.176837


In [21]:
# Búsqueda de hiperparámetros
# ==============================================================================
results = grid_search_forecaster(
              forecaster         = forecaster,
              y                  = data.loc[:end_validation, 'users'],
              exog               = data.loc[:end_validation, exog_features],
              param_grid         = param_grid,
              lags_grid          = lags_grid,
              steps              = 12,
              refit              = False,
              metric             = ['mean_squared_error', 'mean_absolute_error'],
              initial_train_size = len(data.loc[:end_train]),
              method             = 'one_step_ahead',
              fixed_train_size   = False,
              return_best        = True,
              n_jobs             = 'auto',
              verbose            = False,
              show_progress      = True
          )

display(results.head(5))

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = data['users'],
    exog               = data[exog_features],
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data[:end_validation]),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
metric

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\model_selection.py:1436: UserWarning: Models are evaluated using one-step-ahead predictions. This is faster than backtesting the model to predict multiple steps ahead and is useful for quickly comparing different models and hyperparameters, but the results may not reflect the actual performance of the model when predicting multiple steps ahead.
  "than backtesting the model to predict multiple steps ahead and is useful "


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/27 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
  Backtesting metric: 3585.702902258639



,lags,lags_label,params,mean_squared_error,mean_absolute_error,learning_rate,max_depth,n_estimators
0,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",3585.702902,38.861799,0.1,3.0,200.0
1,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",3599.971591,40.186578,0.5,2.0,200.0
2,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",3655.498817,40.811929,0.1,3.0,200.0
3,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.5, 'max_depth': 2, 'n_esti...",3693.968339,41.777409,0.5,2.0,200.0
4,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",3778.541481,40.544159,0.1,3.0,200.0


  0%|          | 0/81 [00:00<?, ?it/s]

,mean_absolute_error
0,57.176837


In [22]:
# Lags grid
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, 'users'],
    exog               = data.loc[:end_validation, exog_features],
    search_space       = search_space,
    steps              = 36,
    refit              = False,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data_train),
    method             = 'backtesting',
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

display(results_search.head(5))

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = data['users'],
    exog               = data[exog_features],
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data[:end_validation]),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
metric

  0%|          | 0/20 [00:00<?, ?it/s]

[W 2024-09-02 23:02:32,711] Trial 15 failed with parameters: {'n_estimators': 900, 'max_depth': 8, 'learning_rate': 0.15918274837202234, 'subsample': 0.7244239328841464, 'colsample_bytree': 0.9830662073260925, 'gamma': 0.3937287275922632, 'reg_alpha': 0.3359906705674698, 'reg_lambda': 0.36958845383485617, 'lags': [1, 2, 3, 23, 24, 25, 167, 168, 169]} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\anaconda\envs\skforecast_13_p12\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\model_selection.py", line 1887, in _objective
    metrics, _ = backtesting_forecaster(
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\model_selection\model_selection.py", line 766, in backtesting_forecaster
    metric_values, back

KeyboardInterrupt: 

In [ ]:
# Lags grid
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]

# Regressor hyperparameters search space
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

results_search, frozen_trial = bayesian_search_forecaster(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, 'users'],
    exog               = data.loc[:end_validation, exog_features],
    search_space       = search_space,
    steps              = 36,
    refit              = False,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data_train),
    method             = 'one_step_ahead',
    fixed_train_size   = False,
    n_trials           = 20,
    random_state       = 123,
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

display(results_search.head(5))

# Backtesting model with exogenous variables on test data
# ==============================================================================
metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = data['users'],
    exog               = data[exog_features],
    steps              = 36,
    metric             = 'mean_absolute_error',
    initial_train_size = len(data[:end_validation]),
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
metric

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1200, 'max_depth': 10, 'learning_rate': 0.022664761670265286, 'subsample': 0.11400878440779741, 'colsample_bytree': 0.9845292840882569, 'gamma': 0.004802254183477734, 'reg_alpha': 0.055560237654553785, 'reg_lambda': 0.22785658420818355}
  Backtesting metric: 40.90867836910601



,lags,params,mean_absolute_error,n_estimators,max_depth,learning_rate,subsample,colsample_bytree,gamma,reg_alpha,reg_lambda
0,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 1200, 'max_depth': 10, 'learn...",40.908678,1200.0,10.0,0.022665,0.114009,0.984529,0.004802,0.055560,0.227857
1,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 1200, 'max_depth': 10, 'learn...",41.424203,1200.0,10.0,0.011347,0.100035,0.950198,0.010401,0.003678,0.196236
2,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 1200, 'max_depth': 10, 'learn...",41.466736,1200.0,10.0,0.012164,0.251910,0.879908,0.043704,0.005815,0.062286
3,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 1200, 'max_depth': 10, 'learn...",41.665886,1200.0,10.0,0.056796,0.107255,0.989298,0.006453,0.003838,0.240537
4,"[1, 2, 3, 23, 24, 25, 167, 168, 169]","{'n_estimators': 1000, 'max_depth': 6, 'learni...",44.160722,1000.0,6.0,0.069081,0.458240,0.764196,0.182492,0.175452,0.531551


  0%|          | 0/81 [00:00<?, ?it/s]

,mean_absolute_error
0,54.767118
